<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
r = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
s = BeautifulSoup(r.text,'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
s.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = s.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[9]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-7"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-7"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">78
</th>


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []
p = first_launch_table.find_all('th')
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


In [11]:
x = []

In [12]:
for i in p:
    j = i.text
    x.append(j)

In [13]:
for j in x:
    if len(j)!=3 and len(j)!=4 :
        s = j[0 : len(j)-1]
        column_names.append(s)

In [14]:
column_names

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster[b]',
 'Launchsite',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

Check the extracted column names


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [15]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date andtime (UTC)']
# Let's initial the launch_dict with each value to be an empty list


In [16]:
launch_dict['Flight No.'] = []
launch_dict['Launchsite'] = []
launch_dict['Payload[c]'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launchoutcome'] = []
# Added some new columns
launch_dict['Version,Booster[b]']=[]
launch_dict['Boosterlanding']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [17]:
launch_dict

{'Flight No.': [],
 'Version,Booster[b]': [],
 'Launchsite': [],
 'Payload[c]': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launchoutcome': [],
 'Boosterlanding': [],
 'Date': [],
 'Time': []}

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [18]:
for rows in first_launch_table.find_all('tr'):
    i = 0
    for columns in rows.find_all('td'):
        #print(columns.text)
        #print("-----------------")
        if len(columns.text)<100:
            if i == 0:
                z = columns.text[0 : len(columns.text)-1]
                #print(z)
                x,y = z.split("2020")
                a = x + "2020"
                launch_dict['Date'].append(a)
                launch_dict['Time'].append(y)
                #launch_dict['Date andtime (UTC)'].append(columns.text[0 : len(columns.text)-1])
            elif i == 1:
                launch_dict['Version,Booster[b]'].append(columns.text[0 : len(columns.text)-1])
            elif i == 2:
                launch_dict['Launchsite'].append(columns.text[0 : len(columns.text)-1])
            elif i == 3:
                launch_dict['Payload[c]'].append(columns.text[0 : len(columns.text)-1])
            elif i == 4:
                launch_dict['Payload mass'].append(columns.text[0 : len(columns.text)-1])
            elif i == 5:
                launch_dict['Orbit'].append(columns.text[0 : len(columns.text)-1])
            elif i == 6:
                launch_dict['Customer'].append(columns.text[0 : len(columns.text)-1])
            elif i == 7:
                launch_dict['Launchoutcome'].append(columns.text[0 : len(columns.text)-1])
            elif i == 8:
                launch_dict['Boosterlanding'].append(columns.text[0 : len(columns.text)-1])
        i = i + 1


In [19]:
s = []

In [20]:
for rows in first_launch_table.find_all('tr'):
    for columns in rows.find_all('th'):
        s.append(columns.text[0:len(columns.text)-1])

In [21]:
launch_dict['Flight No.'] = s[10:]

In [22]:
launch_dict

{'Flight No.': ['78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103'],
 'Version,Booster[b]': ['F9 B5 ♺ B1049.4',
  'F9 B5 ♺ B1046.4',
  'F9 B5 ♺B1051.3',
  'F9 B5 ♺B1056.4',
  'F9 B5 ♺B1059.2',
  'F9 B5 ♺ B1048.5',
  'F9 B5 ♺B1051.4',
  'F9 B5B1058.1[518]',
  'F9 B5 ♺B1049.5',
  'F9 B5 ♺B1059.3',
  'F9 B5B1060.1',
  'F9 B5 ♺ B1058.2[544]',
  'F9 B5 ♺B1051.5',
  'F9 B5 ♺B1049.6[544]',
  'F9 B5 ♺B1059.4',
  'F9 B5 ♺B1060.2[563]',
  'F9 B5 ♺B1058.3[565]',
  'F9 B5 ♺B1051.6[568]',
  'F9 B5 ♺B1060.3',
  'F9 B5B1062.1',
  'F9 B5B1061.1[582]',
  'F9 B5B1063.1',
  'F9 B5 ♺B1049.7[590]',
  'F9 B5 ♺B1058.4[592]',
  'F9 B5 ♺B1051.7',
  'F9 B5 ♺B1059.5'],
 'Launchsite': ['CCAFS,SLC-40',
  'KSC,LC-39A',
  'CCAFS,SLC-40',
  'CCAFS,SLC-40',
  'CCAFS,SLC-40',
  'KSC,LC-39A',
  'KSC,LC-39A',
  'KSC,LC-39A',
  'CCAFS,SLC-40',
  'CCAFS,SLC-40',
  'CC

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [23]:
df=pd.DataFrame(launch_dict)

In [24]:
df.head(26)

,Flight No.,"Version,Booster[b]",Launchsite,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,Date,Time
0,78,F9 B5 ♺ B1049.4,"CCAFS,SLC-40",Starlink 2 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Success(drone ship),7 January 2020,",02:19:21[492]"
1,79,F9 B5 ♺ B1046.4,"KSC,LC-39A",Crew Dragon in-flight abort test[495](Dragon C...,"12,050 kg (26,570 lb)",Sub-orbital[496],NASA (CTS)[497],Success,No attempt,19 January 2020,",15:30[494]"
2,80,F9 B5 ♺B1051.3,"CCAFS,SLC-40",Starlink 3 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Success(drone ship),29 January 2020,",14:07[501]"
3,81,F9 B5 ♺B1056.4,"CCAFS,SLC-40",Starlink 4 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Failure(drone ship),17 February 2020,",15:05[503]"
4,82,F9 B5 ♺B1059.2,"CCAFS,SLC-40",SpaceX CRS-20(Dragon C112.3 ♺),"1,977 kg (4,359 lb)[507]",LEO (ISS),NASA (CRS),Success,Success(ground pad),7 March 2020,",04:50[506]"
5,83,F9 B5 ♺ B1048.5,"KSC,LC-39A",Starlink 5 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Failure(drone ship),18 March 2020,",12:16[510]"
6,84,F9 B5 ♺B1051.4,"KSC,LC-39A",Starlink 6 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Success(drone ship),22 April 2020,",19:30[514]"
7,85,F9 B5B1058.1[518],"KSC,LC-39A",Crew Dragon Demo-2[414](Crew Dragon C206.1 End...,"12,530 kg (27,620 lb)[519]",LEO (ISS),NASA (CCDev),Success,Success(drone ship),30 May 2020,",19:22[517]"
8,86,F9 B5 ♺B1049.5,"CCAFS,SLC-40",Starlink 7 v1.0 (60 satellites),"15,600 kg (34,400 lb)[5]",LEO,SpaceX,Success,Success(drone ship),4 June 2020,",01:25[521]"
9,87,F9 B5 ♺B1059.3,"CCAFS,SLC-40","Starlink 8 v1.0 (58 satellites),[524][525]SkyS...","15,410 kg (33,970 lb)[523]",LEO,SpaceXPlanet Labs,Success,Success(drone ship),13 June 2020,",09:21[523]"


In [25]:
df.to_csv('spacex_web_scraped.csv', index=False)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
